<a href="https://colab.research.google.com/github/preetamjumech/cassandra_practice/blob/main/Preetam_Saha__C22018_07_07_2022_E1_Cassandra_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Cassandra

install https://phoenixnap.com/kb/install-cassandra-on-ubuntu

In [1]:
!apt update > null
!apt install openjdk-8-jdk -y > null
#!java -version
!apt install apt-transport-https > null

In [2]:
!sh -c 'echo "deb http://www.apache.org/dist/cassandra/debian 40x main" > /etc/apt/sources.list.d/cassandra.list'
!wget -q -O - https://www.apache.org/dist/cassandra/KEYS | sudo apt-key add -

OK


In [ ]:
#!cat /etc/apt/sources.list.d/cassandra.list

In [3]:
!apt update > null
!apt install cassandra > null

## Start the Service

In [4]:
#!service cassandra stop
# the ulimit error does not matter
!service cassandra restart

 * Restarting Cassandra cassandra
/etc/init.d/cassandra: 72: ulimit: error setting limit (Operation not permitted)
   ...done.


In [5]:
# keep checking service status until you see [+] next to cassandra
# then wait for at least 60 seconds MORE .... count 1 to 60 if you are feeling impatient
!service  --status-all 

 [ + ]  cassandra
 [ - ]  dbus
 [ ? ]  hwclock.sh
 [ ? ]  kmod
 [ - ]  ntp
 [ - ]  procps
 [ - ]  rsync
 [ - ]  udev
 [ - ]  x11-common


In [6]:
# this will give an error if the cassandra service has not started
# Wait for some time for the service to start
# If required reboot system or stop, restart cassandra service
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  127.0.0.1  69.07 KiB  16      100.0%            abbc1904-76e3-4ec7-a72f-3d0a1779e452  rack1



In [ ]:
# basic test
#!cqlsh

# Basic Commands 
how to execute commands through scripts https://stackoverflow.com/questions/25286253/executing-cql-through-shell-script <br>
some examples https://techfossguru.com/apache-cassandra-python-step-step-guide-ubuntu-example/

In [7]:
!cqlsh -e "help" 


Documented shell commands:
CAPTURE  CLS          COPY  DESCRIBE  EXPAND  LOGIN   SERIAL  SOURCE   UNICODE
CLEAR    CONSISTENCY  DESC  EXIT      HELP    PAGING  SHOW    TRACING

CQL help topics:
AGGREGATES               CREATE_KEYSPACE           DROP_TRIGGER      TEXT     
ALTER_KEYSPACE           CREATE_MATERIALIZED_VIEW  DROP_TYPE         TIME     
ALTER_MATERIALIZED_VIEW  CREATE_ROLE               DROP_USER         TIMESTAMP
ALTER_TABLE              CREATE_TABLE              FUNCTIONS         TRUNCATE 
ALTER_TYPE               CREATE_TRIGGER            GRANT             TYPES    
ALTER_USER               CREATE_TYPE               INSERT            UPDATE   
APPLY                    CREATE_USER               INSERT_JSON       USE      
ASCII                    DATE                      INT               UUID     
BATCH                    DELETE                    JSON            
BEGIN                    DROP_AGGREGATE            KEYWORDS        
BLOB                     DROP_COLUMNF

In [8]:
!cqlsh -e "CREATE KEYSPACE praxis WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };"


Warnings :
Your replication factor 3 for keyspace praxis is higher than the number of nodes 1



In [9]:
# since we are not specifying keyspace separately, we have to prefix table name with keyspace
!cqlsh -e "CREATE TABLE praxis.users (user_id int PRIMARY KEY, first_name text, last_name text );"

In [10]:
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (1, 'Preetam', 'Saha');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (2, 'Rachel', 'Ghosh');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (3, 'Pheebe', 'Das');"

In [11]:
!cqlsh -e "select * from praxis.users;"


 user_id | first_name | last_name
---------+------------+-----------
       1 |    Preetam |      Saha
       2 |     Rachel |     Ghosh
       3 |     Pheebe |       Das

(3 rows)


In [12]:
#another way to specifiy keyspace
!cqlsh -e "use praxis;\
           select * from users;"


 user_id | first_name | last_name
---------+------------+-----------
       1 |    Preetam |      Saha
       2 |     Rachel |     Ghosh
       3 |     Pheebe |       Das

(3 rows)


In [13]:
# easier way to specify keyspace, similar specifying database in mysql
!cqlsh -k praxis -e "select * from users;"


 user_id | first_name | last_name
---------+------------+-----------
       1 |    Preetam |      Saha
       2 |     Rachel |     Ghosh
       3 |     Pheebe |       Das

(3 rows)


# Python
https://docs.datastax.com/en/developer/python-driver/3.25/

In [14]:
!pip install cassandra-driver > null

In [15]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'],port=9042)
session = cluster.connect('praxis',wait_for_all_pools=True)
session.execute('USE praxis')

In [16]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Preetam Saha
2 Rachel Ghosh
3 Pheebe Das


In [17]:
session.execute("INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (4, 'Monica', 'Sen');")

In [18]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Preetam Saha
2 Rachel Ghosh
4 Monica Sen
3 Pheebe Das


In [19]:
session.execute("update praxis.users set first_name = 'Dr Preetam' where user_id = 1;")

In [20]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Dr Preetam Saha
2 Rachel Ghosh
4 Monica Sen
3 Pheebe Das


In [21]:
session.execute("update praxis.users set first_name = 'Mrs Rachel' where first_name = 'Rachel';")

InvalidRequest: ignored

#Tutorial from FreeCodeCamp <br>
https://www.freecodecamp.org/news/the-apache-cassandra-beginner-tutorial/ <br>
alternatively from this [saved file](https://github.com/Praxis-QR/BDSN/blob/main/Documents/The%20Apache%20Cassandra%20Beginner%20Tutorial.pdf)

In [22]:
#!cqlsh -e "CREATE KEYSPACE learn_cassandra WITH REPLICATION = { 'class' : 'NetworkTopologyStrategy','datacenter1' : 3  };"

In [23]:
!cqlsh -k praxis -e "CREATE TABLE users_by_country \
(country text,\
 user_email text,\
 first_name text,\
 last_name text,\
 age smallint,\
 PRIMARY KEY ((country), user_email));"

In [24]:
!cqlsh -k praxis -e "\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age) \
  VALUES('US', 'john@email.com', 'John','Wick',55); \
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'peter@email.com', 'Peter','Clark',65);\
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'bob@email.com', 'Bob','Sandler',23);\
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'alice@email.com', 'Alice','Brown',26);\
"

In [25]:
!cqlsh -k praxis -e "SELECT * FROM users_by_country WHERE country='UK';"


 country | user_email      | age | first_name | last_name
---------+-----------------+-----+------------+-----------
      UK | alice@email.com |  26 |      Alice |     Brown
      UK |   bob@email.com |  23 |        Bob |   Sandler
      UK | peter@email.com |  65 |      Peter |     Clark

(3 rows)


In [26]:
!cqlsh -k praxis -e "drop table users_by_email;"
!cqlsh -k praxis -e "CREATE TABLE users_by_email (\
  user_email text,\
  country text,\
  first_name text,\
  last_name text,\
  age smallint,\
  PRIMARY KEY (user_email));"
  

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.users_by_email' doesn't exist"


In [27]:
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email, country,first_name,last_name,age)  VALUES('john@email.com', 'US', 'John','Wick',55);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('peter@email.com', 'UK', 'Peter','Clark',65);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('bob@email.com', 'UK', 'Bob','Sandler',23);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('alice@email.com', 'UK', 'Alice','Brown',26);"

In [28]:
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE user_email='alice@email.com';"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)


In [29]:
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE age=26 ALLOW FILTERING;"
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE age=26;"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)
<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [30]:
!cqlsh -k praxis -e "CREATE TABLE todo_by_user_email (user_email text,name text,creation_date timestamp,PRIMARY KEY ((user_email), creation_date)) WITH CLUSTERING ORDER BY (creation_date DESC) AND compaction = { 'class' :  'LeveledCompactionStrategy'};"

In [31]:
!cqlsh -k praxis -e "\
CREATE TABLE todos_shared_by_target_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((target_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [32]:
!cqlsh -k praxis -e "\
CREATE TABLE todos_shared_by_source_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((source_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [33]:
!cqlsh -k praxis -e "\
BEGIN BATCH \
  INSERT INTO todo_by_user_email (user_email,creation_date,name) VALUES('alice@email.com', toTimestamp(now()), 'My first todo entry') \
  INSERT INTO todos_shared_by_target_user_email (target_user_email, source_user_email,creation_date,name) VALUES('bob@email.com', 'alice@email.com',toTimestamp(now()), 'My first todo entry') \
  INSERT INTO todos_shared_by_source_user_email (target_user_email, source_user_email,creation_date,name) VALUES('alice@email.com', 'bob@email.com', toTimestamp(now()), 'My first todo entry') \
APPLY BATCH;"

In [34]:
!cqlsh -k praxis -e "SELECT * FROM todos_shared_by_target_user_email WHERE target_user_email='bob@email.com';"


 target_user_email | creation_date                   | name                | source_user_email
-------------------+---------------------------------+---------------------+-------------------
     bob@email.com | 2022-07-07 10:43:32.963000+0000 | My first todo entry |   alice@email.com

(1 rows)


In [35]:
!cqlsh -k praxis -e "SELECT * FROM todo_by_user_email WHERE user_email='alice@email.com';"


 user_email      | creation_date                   | name
-----------------+---------------------------------+---------------------
 alice@email.com | 2022-07-07 10:43:32.962000+0000 | My first todo entry

(1 rows)


#Cassandra Collections <br>
Set, List & Map in CQL with Example <br>
https://www.guru99.com/cassandra-collections-tutorial-set-list-map.html

## Set

In [36]:
!cqlsh -k praxis -e "drop table teacher01;"
!cqlsh -k praxis -e "\
Create table Teacher01 \
( \
id int, \
Name text, \
Email set<text>, \
Primary key(id) \
);\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.teacher01' doesn't exist"


In [37]:
!cqlsh -k praxis  -e "insert into Teacher01(id,Name,Email) \
 values(1,'preetam',{{'abc@gmail.com','xyz@hotmail.com'}});"

In [38]:
!cqlsh -k praxis  -e "insert into Teacher01(id,Name,Email) \
values(2,'rachel',{{'abc1@gmail.com','xyz1@hotmail.com','xyz2@hotmail.com'}});"

In [39]:
!cqlsh -k praxis -e " select * from Teacher01;"


 id | email                                                      | name
----+------------------------------------------------------------+---------
  1 |                       {'abc@gmail.com', 'xyz@hotmail.com'} | preetam
  2 | {'abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com'} |  rachel

(2 rows)


## List

In [40]:
!cqlsh -k praxis -e "drop table teacher02;"
!cqlsh -k praxis -e "\
Create table Teacher02 \
( \
id int, \
Name text, \
Courses list<text>, \
Primary key(id) \
);\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.teacher02' doesn't exist"


In [41]:
!cqlsh -k praxis  -e "insert into Teacher02(id,Name,Courses) values(1,'prithwis',{['FDS','Python']});"
!cqlsh -k praxis  -e "insert into Teacher02(id,Name,Courses) values(2,'charan',{['Comms','English','Interview']});"

In [42]:
!cqlsh -k praxis -e "select * from teacher02;"


 id | courses                           | name
----+-----------------------------------+----------
  1 |                 ['FDS', 'Python'] | prithwis
  2 | ['Comms', 'English', 'Interview'] |   charan

(2 rows)


## Maps

In [43]:
!cqlsh -k praxis -e "drop table if exists courses;"
!cqlsh -k praxis -e "\
Create table courses \
( \
id int, \
Name text, \
Teachers map<text,text>, \
Primary key(id) \
);\
"

In [45]:
!cqlsh -k praxis  -e "insert into courses(id,Name,Teachers) values(100,'PGDM',{{'Stats':'SRC','FDM':'Atanu'}});"
!cqlsh -k praxis  -e "insert into courses(id,Name,Teachers) values(200,'PGDS',{{'Stats':'SRC','ML':'SDG', 'BDSN':'PM'}});"
!cqlsh -k praxis -e "select * from courses;"


 id  | name | teachers
-----+------+---------------------------------------------
 200 | PGDS | {'BDSN': 'PM', 'ML': 'SDG', 'Stats': 'SRC'}
 100 | PGDM |            {'FDM': 'Atanu', 'Stats': 'SRC'}

(2 rows)


In [ ]:
#!cqlsh -k praxis -e "describe Teacher02;"

## Python

In [46]:
rows = session.execute('SELECT * FROM teacher01')
for row in rows:
        print(row)
        emails = row.email
        print(emails)
        emails2 = list(emails)
        for im in emails2:
            print(im)

Row(id=1, email=SortedSet(['abc@gmail.com', 'xyz@hotmail.com']), name='preetam')
SortedSet(['abc@gmail.com', 'xyz@hotmail.com'])
abc@gmail.com
xyz@hotmail.com
Row(id=2, email=SortedSet(['abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com']), name='rachel')
SortedSet(['abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com'])
abc1@gmail.com
xyz1@hotmail.com
xyz2@hotmail.com


In [47]:
rows = session.execute('SELECT * FROM teacher02')
for row in rows:
        print(row)
        courses = row.courses
        print(courses)
        for c in courses:
            print(c)

Row(id=1, courses=['FDS', 'Python'], name='prithwis')
['FDS', 'Python']
FDS
Python
Row(id=2, courses=['Comms', 'English', 'Interview'], name='charan')
['Comms', 'English', 'Interview']
Comms
English
Interview


In [48]:
rows = session.execute('SELECT * FROM courses')
for row in rows:
        print(row)
        print(row.teachers) # this is a dictionary and can be used as such
        for k,v in row.teachers.items():
            print(k,v)
        
        

Row(id=200, name='PGDS', teachers=OrderedMapSerializedKey([('BDSN', 'PM'), ('ML', 'SDG'), ('Stats', 'SRC')]))
{'BDSN': 'PM', 'ML': 'SDG', 'Stats': 'SRC'}
BDSN PM
ML SDG
Stats SRC
Row(id=100, name='PGDM', teachers=OrderedMapSerializedKey([('FDM', 'Atanu'), ('Stats', 'SRC')]))
{'FDM': 'Atanu', 'Stats': 'SRC'}
FDM Atanu
Stats SRC


# Bulk Data Load

In [49]:
!gdown https://drive.google.com/uc?id=1T7teYg1_SXzZLbh8SOF0UIzpyK-L1aZL
!cat Employee.csv

Downloading...
From: https://drive.google.com/uc?id=1T7teYg1_SXzZLbh8SOF0UIzpyK-L1aZL
To: /content/Employee.csv
100% 734/734 [00:00<00:00, 846kB/s]
EmpID,LastName,FirstName,JobDesc,JoinDate,Salary,Comm,DeptID
742866,Bacchan,Amitabh,Executive,2003-03-10,50000,0.1,10
349870,Mukherjee,Rani,Manager,2005-05-04,25000,0.06,40
865477,Dikshit,Madhuri,Clerk,2002-04-04,10000,0.02,20
239456,Khan,Shahrukh,Manager,2004-01-03,30000,0.07,20
897889,Sehwag,Virender,Cus_Rep,2005-01-02,15000,0.05,20
123980,Dhoni,Mahender,Clerk,2004-10-09,9000,0.02,40
822134,Dravid,Rahul,Sr Manager,2000-06-04,40000,0.08,30
997445,Dalmia,Jagmohan,Clerk,2001-07-01,12000,0.02,30
989007,Ganguly,Sourav,Cus_Rep,2002-01-01,20000,0.03,40
299034,Ganesan,Rekha,Director,2002-10-10,60000,0.11,10
546223,Karthikeyan,Narayan,Secretary,2005-12-04,40000,0.09,10
223112,Mirza,Sania,Cus_Rep,2001-11-19,25000,0.04,30

In [50]:
# 
!cqlsh -k praxis -e "drop table if exists emp;"
!cqlsh -k praxis -e "\
    CREATE TABLE emp (\
    empid int PRIMARY KEY, \
    lastname text,\
    firsname text,\
    jobdesc text,\
    joindate text,\
    salary int,\
    comm decimal,\
    deptid text \
    );\
"

In [51]:
!cqlsh -k praxis -e "select * from emp;"


 empid | comm | deptid | firsname | jobdesc | joindate | lastname | salary
-------+------+--------+----------+---------+----------+----------+--------


(0 rows)


In [52]:
!cqlsh -k praxis -e "\
    COPY emp (empid,lastname,firsname,jobdesc,joindate,salary,comm,deptid) \
    FROM 'Employee.csv' \
    WITH HEADER = TRUE;\
"

Using 1 child processes

Starting copy of praxis.emp with columns [empid, lastname, firsname, jobdesc, joindate, salary, comm, deptid].
Processed: 12 rows; Rate:      24 rows/s; Avg. rate:      34 rows/s
12 rows imported from 1 files in 0.355 seconds (0 skipped).


In [53]:
!cqlsh -k praxis -e "select * from emp;"


 empid  | comm | deptid | firsname | jobdesc    | joindate   | lastname    | salary
--------+------+--------+----------+------------+------------+-------------+--------
 546223 | 0.09 |     10 |  Narayan |  Secretary | 2005-12-04 | Karthikeyan |  40000
 223112 | 0.04 |     30 |    Sania |    Cus_Rep | 2001-11-19 |       Mirza |  25000
 989007 | 0.03 |     40 |   Sourav |    Cus_Rep | 2002-01-01 |     Ganguly |  20000
 865477 | 0.02 |     20 |  Madhuri |      Clerk | 2002-04-04 |     Dikshit |  10000
 123980 | 0.02 |     40 | Mahender |      Clerk | 2004-10-09 |       Dhoni |   9000
 897889 | 0.05 |     20 | Virender |    Cus_Rep | 2005-01-02 |      Sehwag |  15000
 349870 | 0.06 |     40 |     Rani |    Manager | 2005-05-04 |   Mukherjee |  25000
 239456 | 0.07 |     20 | Shahrukh |    Manager | 2004-01-03 |        Khan |  30000
 742866 |  0.1 |     10 |  Amitabh |  Executive | 2003-03-10 |     Bacchan |  50000
 997445 | 0.02 |     30 | Jagmohan |      Clerk | 2001-07-01 |      Dalmia

In [54]:
rows = session.execute("SELECT * from emp where jobdesc = 'Cus_Rep'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)

InvalidRequest: ignored

In [55]:
rows = session.execute("SELECT * from emp where empid = 989007")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)

Row(empid=989007, comm=Decimal('0.03'), deptid='40', firsname='Sourav', jobdesc='Cus_Rep', joindate='2002-01-01', lastname='Ganguly', salary=20000)
Ganguly Sourav


In [56]:
#Create index IndexName on KeyspaceName.TableName(ColumnName);
!cqlsh -k praxis -e "create index IndexName on emp (jobdesc);"

In [57]:
rows = session.execute("SELECT * from emp where jobdesc = 'Cus_Rep'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)
        #emails2 = list(emails)
        #for im in emails2:
        #    print(im)

Row(empid=223112, comm=Decimal('0.04'), deptid='30', firsname='Sania', jobdesc='Cus_Rep', joindate='2001-11-19', lastname='Mirza', salary=25000)
Mirza Sania
Row(empid=989007, comm=Decimal('0.03'), deptid='40', firsname='Sourav', jobdesc='Cus_Rep', joindate='2002-01-01', lastname='Ganguly', salary=20000)
Ganguly Sourav
Row(empid=897889, comm=Decimal('0.05'), deptid='20', firsname='Virender', jobdesc='Cus_Rep', joindate='2005-01-02', lastname='Sehwag', salary=15000)
Sehwag Virender


In [58]:
rows = session.execute("SELECT * from emp where jobdesc like '%Manager%'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)
        #emails2 = list(emails)
        #for im in emails2:
        #    print(im)

InvalidRequest: ignored

In [59]:
!cqlsh -k praxis -e "\
CREATE CUSTOM INDEX  fn_prefix ON emp (jobdesc) USING 'org.apache.cassandra.index.sasi.SASIIndex';\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="SASI indexes are disabled. Enable in cassandra.yaml to use."
